<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Big Data II - Armazenamento
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)<br>
Aluno [Rafael Barbosa](https://www.linkedin.com/in/barbosa89/)

---

## Particionamento e orientação à coluna.

Comparação dos métodos de armazenamento, acesso e quantidade de dados escaneados de tabelas de dados não particionada e particionada, utilizando Pandas, parquet e AWS S3, IAM e Athena.

### Download do arquivo `crime.csv` (fonte [Kaggle](https://www.kaggle.com/yamqwe/chicago-crimee?select=chicago_crime_2014.csv)).

In [ ]:
!wget https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/crime.csv -q -O crime.csv
print("crime.csv pronto")

crime.csv pronto


### Carrega os dados em um DataFrame **Pandas** e cria a coluna `reference_date`.

In [ ]:
import pandas as pd
from datetime import datetime

filename = './crime'
df = pd.read_csv(f'./{filename}.csv')

df['reference_date'] = df['Date'].apply(lambda date: datetime.strptime(date.split(sep=' ')[0], '%m/%d/%Y').strftime('%Y-%m-%d'))

df

,index,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,Latitude,Longitude,reference_date
0,0,9446824,HX100141,01/01/2014 02:00:00 AM,0000X W ILLINOIS ST,0460,BATTERY,SIMPLE,STREET,False,False,1831,18,42.0,8.0,08B,41.890828,-87.630235,2014-01-01
1,1,9446748,HX100020,01/01/2014 12:00:00 AM,006XX N DEARBORN ST,0890,THEFT,FROM BUILDING,BAR OR TAVERN,False,False,1832,18,42.0,8.0,06,41.893542,-87.629702,2014-01-01
2,2,9446758,HX100030,01/01/2014 12:30:00 AM,052XX W RACE AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,False,False,1523,15,28.0,25.0,14,41.890046,-87.756333,2014-01-01
3,3,9446760,HX100027,01/01/2014 12:30:00 AM,053XX W WELLINGTON AVE,0460,BATTERY,SIMPLE,APARTMENT,True,False,2514,25,31.0,19.0,08B,41.935011,-87.759739,2014-01-01
4,4,9446764,HX100054,01/01/2014 12:10:00 AM,014XX W LEXINGTON ST,0460,BATTERY,SIMPLE,STREET,False,False,1231,12,25.0,28.0,08B,41.872509,-87.663069,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274418,274418,10362481,HY554205,08/15/2014 09:00:00 PM,048XX N CLARK ST,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,2032,20,46.0,3.0,11,NaN,NaN,2014-08-15
274419,274419,10362558,HY554336,08/31/2014 03:00:00 AM,028XX W 19TH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOSPITAL BUILDING/GROUNDS,False,False,1022,10,12.0,30.0,02,NaN,NaN,2014-08-31
274420,274420,10363233,HY554972,02/20/2014 12:00:00 PM,046XX W NORTH AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,OTHER,False,False,2533,25,37.0,25.0,11,NaN,NaN,2014-02-20
274421,274421,10363368,HY555019,09/02/2014 09:00:00 AM,047XX N BEACON ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,False,False,1913,19,46.0,3.0,11,NaN,NaN,2014-09-02


### Persiste o dado no DataFrame em um arquivo **csv** e em arquivos **parquet**, estes últimos comprimidos e particionados pela coluna `reference_date`.

In [ ]:
### CSV
df.to_csv('./crime_enriched.csv', sep=',', index=False)

### parquet
df.to_parquet('./crime', engine='pyarrow', compression='gzip', partition_cols='reference_date')

print("pronto")

pronto


### Cria recursos no **AWS S3 e AWS IAM** e persista os dados em seus respectivos *buckets* utilizando Python.

- `Buckets` no `AWS S3` para armazenar os arquivos.
![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/s3-buckets.png?raw=true)


- Usuário no `AWS IAM` para fazer o *upload* dos arquivos e suas partições
![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/iam-user.png?raw=true)

In [ ]:
### inserir as credenciais de usuário AWS IAM no Python

from getpass import getpass

print("access_key:")
aws_access_key_id = getpass()

print("secret_key:")
aws_secret_access_key = getpass()

print("chaves salvas")

access_key:
··········
secret_key:
··········
chaves salvas


In [ ]:
### instala SDK Python da AWS
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.4 MB/s eta 0:00:00


In [ ]:
### conecta o usuário AWS

import boto3

client = boto3.client(
  's3',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)

print("acesso garantido")

acesso garantido


In [ ]:
### upload das partições parquet no AWS S3

import os

BUCKET = 'ebac-m41-parquet'

i = 0

for root, dirs, files in os.walk('./crime'):
  elapsed = f'{round(100*i/365, 2)} %'
  print(elapsed)
  for file in files:
    path = os.path.join(root, file)
    bucket_path = '/'.join(path.split(sep='/')[2:])
    client.upload_file(path, BUCKET, bucket_path)
  i = i + 1


0.0 %
0.27 %
0.55 %
0.82 %
1.1 %
1.37 %
1.64 %
1.92 %
2.19 %
2.47 %
2.74 %
3.01 %
3.29 %
3.56 %
3.84 %
4.11 %
4.38 %
4.66 %
4.93 %
5.21 %
5.48 %
5.75 %
6.03 %
6.3 %
6.58 %
6.85 %
7.12 %
7.4 %
7.67 %
7.95 %
8.22 %
8.49 %
8.77 %
9.04 %
9.32 %
9.59 %
9.86 %
10.14 %
10.41 %
10.68 %
10.96 %
11.23 %
11.51 %
11.78 %
12.05 %
12.33 %
12.6 %
12.88 %
13.15 %
13.42 %
13.7 %
13.97 %
14.25 %
14.52 %
14.79 %
15.07 %
15.34 %
15.62 %
15.89 %
16.16 %
16.44 %
16.71 %
16.99 %
17.26 %
17.53 %
17.81 %
18.08 %
18.36 %
18.63 %
18.9 %
19.18 %
19.45 %
19.73 %
20.0 %
20.27 %
20.55 %
20.82 %
21.1 %
21.37 %
21.64 %
21.92 %
22.19 %
22.47 %
22.74 %
23.01 %
23.29 %
23.56 %
23.84 %
24.11 %
24.38 %
24.66 %
24.93 %
25.21 %
25.48 %
25.75 %
26.03 %
26.3 %
26.58 %
26.85 %
27.12 %
27.4 %
27.67 %
27.95 %
28.22 %
28.49 %
28.77 %
29.04 %
29.32 %
29.59 %
29.86 %
30.14 %
30.41 %
30.68 %
30.96 %
31.23 %
31.51 %
31.78 %
32.05 %
32.33 %
32.6 %
32.88 %
33.15 %
33.42 %
33.7 %
33.97 %
34.25 %
34.52 %
34.79 %
35.07 %
35.34 %
35.62 %
35

### Cria recursos no **AWS Athena** para as tabelas dos arquivos `csv` e `parquet`.

- Cria tabela a partir de `csv`

Código SQL utilizadono Athena:
```sql
CREATE EXTERNAL TABLE `crime_csv`(
  `index` bigint,
  `id` string,
  `case number` string,
  `date` string,
  `block` string,
  `iucr` string,
  `primary type` string,
  `description` string,
  `location description` string,
  `arrest` string,
  `domestic` string,
  `beat` string,
  `district` string,
  `ward` string,
  `community area` string,
  `fbi code` string,
  `latitude` string,
  `longitude` string,
  `reference_date` string)
ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.OpenCSVSerde'
 WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '\"',
   'escapeChar' = '\\'
   )
STORED AS INPUTFORMAT
  'org.apache.hadoop.mapred.TextInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  's3://modulo-42-ebac-csv/'
TBLPROPERTIES (
  "skip.header.line.count"="1")
```

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-create-csv.png?raw=true)
![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-csv.png?raw=true)

- Cria tabela a partir de `parquet`

Código SQL utilizadono Athena:
```sql
CREATE EXTERNAL TABLE `crime_parquet`(
  `index` bigint,
  `id` bigint,
  `case number` string,
  `date` string,
  `block` string,
  `iucr` string,
  `primary type` string,
  `description` string,
  `location description` string,
  `arrest` boolean,
  `domestic` boolean,
  `beat` bigint,
  `district` bigint,
  `ward` double,
  `community area` double,
  `fbi code` string,
  `latitude` double,
  `longitude` double)
PARTITIONED BY (
  `reference_date` string)
ROW FORMAT SERDE
  'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
LOCATION
  's3://modulo-42-ebac-parquet/'
```

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-create-parquet.png?raw=true)


- Carrega partições `parquet`
```sql
MSCK REPAIR TABLE `crime_parquet`;
```

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-load-parquet.png?raw=true)
![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-parquet.png?raw=true)

### Executa consultas SQL em ambas as tabelas e observa a **diferença na quantidade de dados escaneados**.

- Efeito da **orientação a coluna**:

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-select-locationdescript-csv.png?raw=true)

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-select-locationdescript-parquet.png?raw=true)

A consulta escaneou 47.34 MB para a tabela `crime_csv`, que é o mesmo tamanho do arquivo, logo um *full scan*. Já para a tabeka `crime_parquet`, a consulta escaneou 0.61 MB. Ou seja, a tabela com o dado **orientado a coluna** escaneou **77 vezes** menos dados para a consulta SQL que seu par em `csv`.

 - Efeito do **particionamento**:

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-select-all-date-csv.png?raw=true)

![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-select-all-date-parquet.png?raw=true)

A consulta escaneou 47.34 MB para a tabela `crime_csv`, que é o mesmo tamanho do arquivo, logo um *full scan*. Já para a tabeka `crime_parquet`, a consulta escaneou 0.98 MB. Ou seja, a tabela com o dado **particionado** escaneou **48 vezes** menos dados para a consulta SQL que seu par em `csv`.

 - Efeito da **orientação a coluna** e do **particionamento**:

 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-select-location-date-csv.png?raw=true)

 ![](https://github.com/rafie-b/Profession-Data-Analyst/blob/main/M41%20images/Athena-select-location-date-parquet.png?raw=true)

A consulta escaneou 47.34 MB para a tabela `crime_csv`, que é o mesmo tamanho do arquivo, logo um *full scan*. Já para a tabeka `crime_parquet`, a consulta escaneou 0.03 MB. Ou seja, a tabela com o dado **particionado** e **orientado a coluna** escaneou **1337 vezes** menos dados para a consulta SQL que seu par em `csv`.